# 生成器（Generator）详解与演示

本教程将详细讲解 PyTorch 中生成器的实现和使用。生成器是生成对抗网络（GAN）中的重要组成部分，其主要作用是将随机噪声转换为有意义的数据表示。

在本例中，我们将实现一个能够生成高维特征向量的生成器，这些向量可以表示句子或其他类型的数据。

## 1. 导入必要的库

首先，我们需要导入 PyTorch 相关的库和可视化工具。

In [2]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns

# 设置随机种子以保证结果可复现
torch.manual_seed(42)

# 设置显示中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False    # 用来正常显示负号

## 2. 定义生成器类

下面我们将定义生成器类。这个生成器包含以下关键组件：
- 全连接层（Linear layers）
- LeakyReLU 激活函数
- Dropout 层用于防止过拟合

生成器的结构是一个前馈神经网络，将低维随机噪声映射到高维特征空间。

In [ ]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, 
                 noise_size,          # 输入噪声的维度（通常是随机向量 z）
                 output_size,         # 输出向量的维度（如句子嵌入维度）
                 hidden_sizes,        # 隐藏层结构
                 dropout_rate):       # dropout 比率，用于防止过拟合
        super(Generator, self).__init__()

        layers = []
        hidden_sizes = [noise_size] + hidden_sizes  # 将输入维度添加到隐藏层列表开头

        # 逐层构建隐藏层：每层包含 Linear -> LeakyReLU -> Dropout
        for i in range(len(hidden_sizes) - 1):
            layers.extend([
                # 全连接层：将上一层的输出转换为当前层的维度
                nn.Linear(hidden_sizes[i], hidden_sizes[i+1]),     
                # LeakyReLU激活函数：，防止梯度消失
                #激活函数的作用是引入非线性，从而让神经网络能够拟合和学习复杂的函数关系
                nn.LeakyReLU(0.2, inplace=True),                   
                # Dropout层：随机丢弃一部分神经元，防止过拟合
                nn.Dropout(dropout_rate)                           
            ])

        # 输出层：最后一层不需要激活函数和Dropout，直接映射到目标维度
        layers.append(nn.Linear(hidden_sizes[-1], output_size))

        # 将所有层组合成一个序列网络
        self.layers = nn.Sequential(*layers)

    def forward(self, noise):
        """
        生成器的前向传播函数
        """
        output_rep = self.layers(noise)  # 通过网络各层进行前向传播
        return output_rep  # 返回生成的向量表示


In [31]:
# 创建生成器实例
generator = Generator(
    noise_size=100,      # 输入噪声维度为100
    output_size=768,     # 输出维度为768（与BERT等预训练模型的隐藏层维度相匹配）
    hidden_sizes=[768,1024,768],  # 三层隐藏层结构：第一层768维->第二层1024维->第三层768维
    dropout_rate=0.1     # 10%的dropout率，在训练时随机丢弃10%的神经元
)

# 打印生成器结构以查看网络架构
print("生成器结构：")
print(generator)

生成器结构：
Generator(
  (layers): Sequential(
    (0): Linear(in_features=100, out_features=768, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=768, out_features=1024, bias=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=1024, out_features=768, bias=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=768, out_features=768, bias=True)
  )
)


## 4. 生成随机噪声并通过生成器

现在我们将生成一些随机噪声向量，并将它们传递给生成器，观察输出结果。我们将生成多个样本以观察输出的分布特征。

In [27]:
# 生成一批随机噪声
batch_size = 32
noise = torch.randn(batch_size, 100)  # 生成10个100维的随机向量
print("\n输入噪声（z）的形状：", noise.shape)

# 将噪声传入生成器
with torch.no_grad():  # 不需要计算梯度
    generated_vectors = generator(noise)

# 打印生成的向量的形状和其中一个样本
print(f"生成向量的形状：{generated_vectors.shape}")
print("\n生成向量的示例")
print(generated_vectors)


输入噪声（z）的形状： torch.Size([32, 100])
生成向量的形状：torch.Size([32, 768])

生成向量的示例
tensor([[-0.0031,  0.0289,  0.0267,  ..., -0.0121,  0.0810,  0.0217],
        [-0.1185, -0.0718,  0.1118,  ..., -0.0588,  0.0549,  0.0486],
        [ 0.0148, -0.0003, -0.0515,  ...,  0.0420,  0.0493,  0.0519],
        ...,
        [ 0.0201, -0.0937, -0.0630,  ...,  0.0018,  0.0370,  0.0550],
        [ 0.0267, -0.0242, -0.0504,  ..., -0.0425,  0.0114, -0.0273],
        [ 0.0090, -0.0644, -0.0105,  ...,  0.0634,  0.0039,  0.0196]])
